In [ ]:
!pip install tensorflow numpy opencv-python-headless

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install colab-convert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for colab-convert: filename=colab_convert-2.0.5-py3-none-any.whl size=19223 sha256=df81b9b5a5b4ff9194a33e64227a02ff3f4ead4cbfc9212489b5b6cab4c9f48e
  Stored in directory: /root/.cache/pip/wheels/3d/1d/81/3103aec20b94fd2870896e0fb56130af08ff64e34c3c3bec76
Successfully built colab-convert


In [ ]:
pip install --upgrade tensorflow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#from tensorflow.keras.layers import LeakyReLU


In [ ]:
pip install keras-applications


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.0 MB/s eta 0:00:00


In [ ]:
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg

--2023-04-27 10:13:02--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2023-04-27 10:13:02 (62.2 MB/s) - ‘yolov3.cfg’ saved [8342/8342]



In [ ]:
import urllib.request
import os

# Download YOLOv3 weights file
if not os.path.exists('yolov3.weights'):
    urllib.request.urlretrieve('https://pjreddie.com/media/files/yolov3.weights', 'yolov3.weights')

# Convert YOLOv3 weights file to Keras model file
if not os.path.exists('yolov3.h5'):
    !python convert.py yolov3.cfg yolov3.weights yolov3.h5


2023-04-27 10:13:07.169794: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 10:13:08.278715: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.
Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)
Parsing section convolutional_1
conv2d bn leaky (3, 3, 32, 64)
Parsing section convolutional_2
conv2d bn leaky (1, 1, 64, 32)
Parsing section convolutional_3
conv2d bn leaky (3, 3, 32, 64)
Parsing section shortcut_0
Parsing section convolutional_4
conv2d bn leaky (3, 3, 64, 128)
Parsing section convolutional_5
conv2d bn leaky (1, 1, 128, 64)
Parsing section convolutional_6
conv2d 

In [ ]:
import tensorflow as tf

# Load YOLOv3 model from Keras model file
model = tf.keras.models.load_model('yolov3.h5', compile=False)

# Freeze pre-trained layers
for layer in model.layers[:-3]:
    layer.trainable = False


In [ ]:
import tensorflow as tf

# define input layers
input1 = tf.keras.layers.Input(shape=(None, None, 3))
input2 = tf.keras.layers.Input(shape=(None, None, 3))
input3 = tf.keras.layers.Input(shape=(None, None, 3))

# concatenate inputs into a single tensor
inputs = tf.keras.layers.concatenate([input1, input2, input3])

# apply global average pooling layer
x = tf.keras.layers.GlobalAveragePooling2D()(inputs)

# add custom layers for fine-tuning
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# create fine-tuned model
fine_tuned_model = tf.keras.models.Model(inputs=[input1, input2, input3], outputs=x)


In [ ]:
#!git clone https://github.com/qqwweee/keras-yolo3.git


In [ ]:
# Specify input shape
input_shape = (416, 416, 3)
fine_tuned_model.build(input_shape)


In [ ]:
# Compile the model with MSE as the loss function
fine_tuned_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])


In [ ]:
# Compile the model
#fine_tuned_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import requests

# Define URLs of images to download
image_urls = ['https://st.depositphotos.com/1031223/3245/i/600/depositphotos_32451819-stock-photo-stacked-sugar-bag.jpg',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRC2jZ3ayVLhRfbzvyT5Tcl35tAC365DIcWBrARv2voouf5TAlZ-c-ahqYOpZt_4Cs7u-0&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRuYOUlll_S14gHoh8sW4iJuqY1qXrL9mgCkDNfJGQW1AqA_OOy0IYrJGQ2It6vhOGgSKo&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ7hqdS8heGlbvMCmu5RU40P043PTp5ONK422QQq76o9VnrQdf22WGnwMWKlKaF_MLfpxc&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQbt0EdJAmYMK67aEPwqeD-mYj4I3aSnfn-lg&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTraWDxHChvgXtsv6dHoz2Gq_D5pkXdya88dA&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRt-05UOW9g8u_DQvwlkC_8uK456CA2I588Stgsx6nJ6IZm6TDOS5xMEh_12OAGqpU5GI4&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTRpLPfDyMqzYpnJ8wh4WAccM1ka_Ki2BCmj47naEI7fLIo6rRmCSdUcFVkDjZz0fsblKo&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQtlPtQYmPv2uCYDWQ9lDouXlCspAAkWMxz3Q&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRQp2jjUqVGyvRQKHW5y3BF2hS4hcY0i66v0Q&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSjVyKBef5wpYcu6RttCkJcdGeYciSEInsEZHFA04yW10der7ZfGoouz_Mp8ve5NrmjBXY&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQc5uQ8FsUb-Afb3LjQKLm1tgUW6usIDw7b0U9NGyD7EPzf5BlWrhKOEdOLEOoYfd2j328&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQsxHnaizrrZ5FAF0mKCEO9VFG4MMxlZTcsGApUKcj4cxHx2WcjcZZkZM-YLLWATgnPUVM&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS6qvFLj9RK8NB73dqNmex2gCk0PUOPlC2bcCn0Lm89IBEdeOdGLD_5HWkUwYzjIZAUOzY&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSXLwxw5X2-vZ1KYT_mO2V-hzfhBqjmQHDgvdCCPKSq2HLR5sDkSyI_xduaFNLZch37fmQ&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRoWZ6cHeYPpISOYGQDXRJ3qV8KHE48vGGKiIZ6iGZe2eaRhUUxays4n9CDMNXHKIFLhjo&usqp=CAU',
'https://thumbs.dreamstime.com/b/bags-sugar-freight-trolley-211009763.jpg',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSZPSK1vc_IVFiCy_qVJPQ2cVxch3Xi-7rOOs5ZIG85q9yyPRwz6JKkSuSDwMovDuM8qrQ&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSckDz4KRVNvxSIN0VIbKBgxGMPb-QSNguIcBM3yjemZtkpUAFYh6Jt6KwzZI3A71MPlhM&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQX7qUEIVYmdLcJKlmsg-wDtnEjGcwBbhWrOg&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRRcyNT8neiDIOKFUzOxjdQ8xm3eT9efOfs8cA9IT0U5QsgF1BSZRZkRmMHxKo4GqNMkdo&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQN0UqyrLeOmeRbqPdENfofAqYmrma2NtCIb10wFy1YGUNG044vmKGXWxlf-Zf_yNeMUX8&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRlCfiIdTBlkH_VhZ4AjCmwPRH4U3wYSGdnwQBgnR-v359gwRV-KgPRGszakEj5RnNZppk&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSJTFo-sjy1-WEk5bYEAzbAFrIMW_3GYKwwM1UyCtb7EQre9aIhHLeOG4IJ8Xdxvu67ZNk&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQNA3BW8LuhzRR1089bWok7pC4xclgq0n43UA&usqp=CAU',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS4Bi5D8Tjx6vN10gYjRscO0p_hsVdn7usKLQ&usqp=CAU'
]

# Create directory for images (if it doesn't already exist)
if not os.path.exists("images"):
    os.makedirs("images")

# Download and save images
for i, url in enumerate(image_urls):
    try:
        # Send GET request for image
        response = requests.get(url)
        # Write image to file
        with open(f"images/sack_{i+1}.jpg", "wb") as f:
            f.write(response.content)
    except:
        print(f"Could not download image from {url}")


In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
'''
# Set path to CSV file and image directory
annotations_path = "annotations.csv"
image_dir = "images/"

# Load annotations from CSV file
annotations = pd.read_csv(annotations_path)

# Create training dataset
X_train = []
y_train = []
for i, row in annotations.iterrows():
    # Load image
    img_path = os.path.join(image_dir, row['filename'])
    img = cv2.imread(img_path)
    if img is not None:
        # Resize image to (416, 416)
        img = cv2.resize(img, (416, 416))
        # Convert image to RGB format
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Normalize pixel values to [0, 1]
        img = img / 255.0
        # Add image to training dataset
        X_train.append(img)
        y_train.append(row['label'])
    else:
        print(f"Could not read image {img_path}")

X_train = np.array(X_train)
y_train = np.array(y_train)
'''

'\n# Set path to CSV file and image directory\nannotations_path = "annotations.csv"\nimage_dir = "images/"\n\n# Load annotations from CSV file\nannotations = pd.read_csv(annotations_path)\n\n# Create training dataset\nX_train = []\ny_train = []\nfor i, row in annotations.iterrows():\n    # Load image\n    img_path = os.path.join(image_dir, row[\'filename\'])\n    img = cv2.imread(img_path)\n    if img is not None:\n        # Resize image to (416, 416)\n        img = cv2.resize(img, (416, 416))\n        # Convert image to RGB format\n        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)\n        # Normalize pixel values to [0, 1]\n        img = img / 255.0\n        # Add image to training dataset\n        X_train.append(img)\n        y_train.append(row[\'label\'])\n    else:\n        print(f"Could not read image {img_path}")\n\nX_train = np.array(X_train)\ny_train = np.array(y_train)\n'

In [ ]:
# Set path to CSV file and image directory
annotations_path = "annotations.csv"
image_dir = "images/"

# Load annotations from CSV file
annotations = pd.read_csv(annotations_path)

# Create training dataset
X_train = []
y_train = []
for i, row in annotations.iterrows():
    # Load image
    img_path = os.path.join(image_dir, row['filename'])
    img = cv2.imread(img_path)
    if img is not None:
        # Resize image to (416, 416)
        img = cv2.resize(img, (416, 416))
        # Convert image to RGB format
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Normalize pixel values to [0, 1]
        img = img / 255.0
        # Add image to training dataset
        X_train.append([img, img, img])  # repeat the same image 3 times
        y_train.append(row['label'])
    else:
        print(f"Could not read image {img_path}")

X_train = [np.array(X_train)[:, i] for i in range(3)]  # separate the 3 images
y_train = np.array(y_train)

Could not read image images/sack_0.jpg


In [ ]:
X_train

[array([[[[0.80784314, 0.78039216, 0.6745098 ],
          [0.82745098, 0.8       , 0.69411765],
          [0.84313725, 0.81568627, 0.70980392],
          ...,
          [0.87843137, 0.85098039, 0.8       ],
          [0.87843137, 0.85098039, 0.8       ],
          [0.87843137, 0.85098039, 0.8       ]],
 
         [[0.80392157, 0.76862745, 0.6627451 ],
          [0.82352941, 0.78823529, 0.68235294],
          [0.84313725, 0.80392157, 0.70196078],
          ...,
          [0.87843137, 0.85098039, 0.8       ],
          [0.87843137, 0.85098039, 0.8       ],
          [0.87843137, 0.85098039, 0.8       ]],
 
         [[0.79607843, 0.75294118, 0.64313725],
          [0.81176471, 0.76862745, 0.6627451 ],
          [0.82745098, 0.78431373, 0.6745098 ],
          ...,
          [0.87843137, 0.85098039, 0.8       ],
          [0.87843137, 0.85098039, 0.8       ],
          [0.87843137, 0.85098039, 0.8       ]],
 
         ...,
 
         [[0.90196078, 0.89019608, 0.90196078],
          [0.90588

In [ ]:
y_train

array(['sack', 'sack', 'sack', 'sack', 'sack', 'sack', 'sack', 'sack',
       'sack', 'sack', 'sack', 'sack', 'sack', 'sack', 'sack', 'sack',
       'sack', 'sack', 'sack', 'sack', 'sack', 'sack', 'sack', 'sack',
       'sack'], dtype='<U4')

In [ ]:
fine_tuned_model.fit(X_train, y_train, batch_size=16, epochs=10)

In [ ]:
annotations_test = pd.read_csv('annotations_test.csv')
X_test = []
y_test = []
for i, row in annotations_test.iterrows():
    img = cv2.imread(row['filename'])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (416, 416))
    img = img / 255.0
    X_test.append(img)
    y_test.append(row['num_bags'])
X_test = np.array(X_test)
y_test = np.array(y_test)

results = fine_tuned_model.evaluate(X_test, y_test)
print('Test Loss:', results[0])
print('Test Accuracy:', results[1])


In [ ]:
fine_tuned_model.save('bag_estimator_model.h5')